<a href="https://colab.research.google.com/github/YuriySamorodov/MIPT/blob/main/%D0%A1%D0%B5%D0%BC%D0%B5%D1%81%D1%82%D1%80%202/Deep%20Learning/mipt_bio_2025_yuriysamorodov_torch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Включаем GPU:

**Google Colab**

Среда выполнения > Сменить среду выполнения > T4 GPU

**Kaggle**

Settings > Accelerator > GPU P100



In [ ]:
# Установка зависимостей
# ETA ~ 1 минута 45 сек.
!pip install pandas numpy torch tqdm torchvision --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 110.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 98.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 46.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 106.7 MB/s eta 0:00:00


In [ ]:
# Импорт библиотек
import torch
from tqdm import tqdm
import pandas as pd
import numpy as np
from torch.utils.data import Dataset, DataLoader
from torch import nn, optim
from torchvision import transforms
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [ ]:
# Проверка GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Используемое устройство: {device}")

Используемое устройство: cuda


In [ ]:
# Загрузка данных
train_df = pd.read_csv('fmnist_train.csv')
test_df = pd.read_csv('fmnist_test.csv')

# Обработка NaN
print(f"Пропуски в тренировочных данных: {train_df.isna().sum().sum()}")
print(f"Пропуски в тестовых данных: {test_df.isna().sum().sum()}")
train_df = train_df.dropna()

# Преобразование в тензоры
class FashionMNISTDataset(Dataset):
    def __init__(self, df, train=True):
        self.train = train
        if self.train:
            self.labels = torch.tensor(df['label'].values, dtype=torch.long)
            self.features = torch.tensor(df.drop(['label', 'Id'], axis=1).values,
                                       dtype=torch.float32).view(-1, 1, 28, 28) / 255.0
        else:
            self.features = torch.tensor(df.drop('Id', axis=1).values,
                                       dtype=torch.float32).view(-1, 1, 28, 28) / 255.0
            self.ids = df['Id'].values

    def __len__(self):
        return len(self.features)

    def __getitem__(self, idx):
        if self.train:
            return self.features[idx], self.labels[idx]
        return self.features[idx], self.ids[idx]

# Создание датасетов
train_dataset = FashionMNISTDataset(train_df)
test_dataset = FashionMNISTDataset(test_df, train=False)

Пропуски в тренировочных данных: 424
Пропуски в тестовых данных: 0


In [ ]:
# Архитектура моделей

class CNNModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.model = nn.Sequential(
            nn.Conv2d(1, 32, 3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.Conv2d(32, 32, 3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Dropout(0.25),

            nn.Conv2d(32, 64, 3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.Conv2d(64, 64, 3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Dropout(0.25),

            nn.Flatten(),
            nn.Linear(64*7*7, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(512, 10)
        )

    def forward(self, x):
        return self.model(x)

In [ ]:
# Обучение модели

# Параметры
BATCH_SIZE = 128
EPOCHS = 50
LR = 0.001

# Даталоадеры
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)

# Инициализация модели
model = CNNModel().to(device)
optimizer = optim.Adam(model.parameters(), lr=LR)
criterion = nn.CrossEntropyLoss()

# Цикл обучения
for epoch in range(EPOCHS):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    # Прогресс-бар с описанием
    progress_bar = tqdm(train_loader, desc=f'Epoch {epoch+1}/{EPOCHS}',
                        bar_format='{l_bar}{bar:20}{r_bar}{bar:-20b}')

    for batch_idx, (inputs, labels) in enumerate(progress_bar):
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # Расчет точности
        preds = torch.argmax(outputs, dim=1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)
        running_loss += loss.item()

        # Обновление прогресс-бара
        progress_bar.set_postfix({
            'Loss': f'{loss.item():.4f}',
            'Accuracy': f'{100 * correct / total:.2f}%'
        })

    # Статистика эпохи
    epoch_loss = running_loss / len(train_loader)
    epoch_acc = correct / total
    print(f'Epoch {epoch+1} Summary:')
    print(f'Avg Loss: {epoch_loss:.4f} | Accuracy: {epoch_acc*100:.2f}%')
    print('-'*50)



Epoch 1/50: 100%|████████████████████| 134/134 [00:02<00:00, 46.70it/s, Loss=0.4042, Acc=80.64%]


Epoch 1 Summary:
Avg Loss: 0.5298 | Accuracy: 80.64%
--------------------------------------------------


Epoch 2/50: 100%|████████████████████| 134/134 [00:01<00:00, 82.30it/s, Loss=0.9092, Acc=87.44%]


Epoch 2 Summary:
Avg Loss: 0.3461 | Accuracy: 87.44%
--------------------------------------------------


Epoch 3/50: 100%|████████████████████| 134/134 [00:01<00:00, 82.50it/s, Loss=0.5101, Acc=88.77%]


...

Epoch 48 Summary:
Avg Loss: 0.0302 | Accuracy: 98.91%
--------------------------------------------------


Epoch 49/50: 100%|████████████████████| 134/134 [00:01<00:00, 79.45it/s, Loss=0.1117, Acc=98.90%]


Epoch 49 Summary:
Avg Loss: 0.0303 | Accuracy: 98.90%
--------------------------------------------------


Epoch 50/50: 100%|████████████████████| 134/134 [00:01<00:00, 78.82it/s, Loss=0.0203, Acc=98.82%]

Epoch 50 Summary:
Avg Loss: 0.0313 | Accuracy: 98.82%
--------------------------------------------------


In [ ]:
# Предсказание и сохранение результатов

model.eval()
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

predictions = []
ids_list = []
with torch.no_grad():
    for inputs, ids in test_loader:
        inputs = inputs.to(device)
        outputs = model(inputs)
        preds = torch.argmax(outputs, dim=1)
        predictions.extend(preds.cpu().numpy())
        ids_list.extend(ids.numpy())

# Создание submission файла
submission = pd.DataFrame({'Id': ids_list, 'label': predictions})
submission.to_csv('submission.csv', index=False)
print('Файл submission.csv успешно создан!')


Файл submission.csv успешно создан!
